In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from lightgbm import LGBMClassifier as lgb
from sklearn.metrics import roc_auc_score, roc_curve
from matplotlib import pyplot as plt
np.set_printoptions( threshold=100 )

In [ ]:
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
# features = pd.read_csv('/kaggle/input/jane-street-market-prediction/features.csv')
# example_sample_submission   = pd.read_csv('/kaggle/input/jane-street-market-prediction/example_sample_submission.csv')


In [ ]:
example_test   = pd.read_csv('/kaggle/input/jane-street-market-prediction/example_test.csv')

## Data Understanding

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
train.groupby('date')['ts_id'].count().plot(kind='bar')

In [ ]:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [ ]:
missing_values_table(train)

In [ ]:
# Get the columns with > 50% missing
missing_df = missing_values_table(train);
missing_columns = list(missing_df[missing_df['% of Total Values'] > 50].index)
print('We will remove %d columns.' % len(missing_columns))

In [ ]:
#train_ = df.drop(list(missing_columns), axis=1)


## Data Prep

In [ ]:
train = train.query('weight > 0').reset_index(drop = True)

train['action'] =  (  (train['resp_1'] > 0 ) & (train['resp_2'] > 0 ) & (train['resp_3'] > 0 ) & (train['resp_4'] > 0 ) &  (train['resp'] > 0 )   ).astype('int')
features = [c for c in train.columns if 'feature' in c]

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

dates = train.date
X = train[features]
# X = X.fillna(X.mean())
# y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T #Multitarget
y = train['action'].values

f_mean = np.mean(train[features[1:]].values,axis=0)

In [ ]:
del(train)

In [ ]:
from sklearn.impute import SimpleImputer

fill_NaN = SimpleImputer(missing_values=np.nan, strategy='mean')
columns = X.columns
X = pd.DataFrame(fill_NaN.fit_transform(X))
X.columns = columns
X.index = X.index


# imp = SimpleImputer(missing_values=np.nan, strategy='mean')
# imp.fit(X)
X.head()


In [ ]:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns(X)

In [ ]:
# import numpy as np
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer

# imp = IterativeImputer(max_iter=10, random_state=0)
# imp.fit(X)

# # the model learns that the second feature is double the first
# print(np.round(imp.transform(X)))


In [ ]:
## standartization'
from sklearn.preprocessing import StandardScaler

# create a scaler object
scaler = StandardScaler()
columns = X.columns
# fit and transform the data
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X.columns = columns
X

In [ ]:
## build feature 
# import sklearn.preprocessing as preproc
# X = preproc.PolynomialFeatures(include_bias=False).fit_transform(X)

In [ ]:
# train.ts_id.nunique()

In [ ]:
#lags
# X['date'] = dates
# X = X.set_index(["date"]).shift(1)
# X = X.fillna(0)
# X.head()

# difs 

X= X.diff().fillna(0)


In [ ]:
df_sample = X.copy()
df_sample['action'] = y.copy()

df_sample = df_sample.sample(frac=0.5, replace=False, random_state=1)

In [ ]:
# import seaborn as sns
# sns.pairplot(df_sample)

In [ ]:
# downsampled

## Model

In [ ]:
"""
This module provides a class to split time-series data for back-testing and evaluation.
The aim was to extend the current sklearn implementation and extend it's uses.

Might be useful for some ;)
"""

import logging
from typing import Optional

import numpy as np
from sklearn.model_selection._split import _BaseKFold
from sklearn.utils import indexable
from sklearn.utils.validation import _num_samples

LOGGER = logging.getLogger(__name__)


class TimeSeriesSplit(_BaseKFold):  # pylint: disable=abstract-method
    """Time Series cross-validator

    Provides train/test indices to split time series data samples that are observed at fixed time intervals,
    in train/test sets. In each split, test indices must be higher than before, and thus shuffling in cross validator is
    inappropriate.

    This cross_validation object is a variation of :class:`TimeSeriesSplit` from the popular scikit-learn package.
    It extends its base functionality to allow for expanding windows, and rolling windows with configurable train and
    test sizes and delays between each. i.e. train on weeks 1-8, skip week 9, predict week 10-11.

    In this implementation we specifically force the test size to be equal across all splits.

    Expanding Window:

            Idx / Time  0..............................................n
            1           |  train  | delay |  test  |                   |
            2           |       train     | delay  |  test  |          |
            ...         |                                              |
            last        |            train            | delay |  test  |

    Rolling Windows:
            Idx / Time  0..............................................n
            1           | train   | delay |  test  |                   |
            2           | step |  train  | delay |  test  |            |
            ...         |                                              |
            last        | step | ... | step |  train  | delay |  test  |

    Parameters:
        n_splits : int, default=5
            Number of splits. Must be at least 4.

        train_size : int, optional
            Size for a single training set.

        test_size : int, optional, must be positive
            Size of a single testing set

        delay : int, default=0, must be positive
            Number of index shifts to make between train and test sets
            e.g,
            delay=0
                TRAIN: [0 1 2 3] TEST: [4]
            delay=1
                TRAIN: [0 1 2 3] TEST: [5]
            delay=2
                TRAIN: [0 1 2 3] TEST: [6]

        force_step_size : int, optional
            Ignore split logic and force the training data to shift by the step size forward for n_splits
            e.g
            TRAIN: [ 0  1  2  3] TEST: [4]
            TRAIN: [ 0  1  2  3  4] TEST: [5]
            TRAIN: [ 0  1  2  3  4  5] TEST: [6]
            TRAIN: [ 0  1  2  3  4  5  6] TEST: [7]

    Examples
    --------
    >>> X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
    >>> y = np.array([1, 2, 3, 4, 5, 6])
    >>> tscv = TimeSeriesSplit(n_splits=5)
    >>> print(tscv)  # doctest: +NORMALIZE_WHITESPACE
    TimeSeriesSplit(train_size=None, n_splits=5)
    >>> for train_index, test_index in tscv.split(X):
    ...    print('TRAIN:', train_index, 'TEST:', test_index)
    ...    X_train, X_test = X[train_index], X[test_index]
    ...    y_train, y_test = y[train_index], y[test_index]
    TRAIN: [0] TEST: [1]
    TRAIN: [0 1] TEST: [2]
    TRAIN: [0 1 2] TEST: [3]
    TRAIN: [0 1 2 3] TEST: [4]
    TRAIN: [0 1 2 3 4] TEST: [5]
    """

    def __init__(self,
                 n_splits: Optional[int] = 5,
                 train_size: Optional[int] = None,
                 test_size: Optional[int] = None,
                 delay: int = 0,
                 force_step_size: Optional[int] = None):

        if n_splits and n_splits < 5:
            raise ValueError(f'Cannot have n_splits less than 5 (n_splits={n_splits})')
        super().__init__(n_splits, shuffle=False, random_state=None)

        self.train_size = train_size

        if test_size and test_size < 0:
            raise ValueError(f'Cannot have negative values of test_size (test_size={test_size})')
        self.test_size = test_size

        if delay < 0:
            raise ValueError(f'Cannot have negative values of delay (delay={delay})')
        self.delay = delay

        if force_step_size and force_step_size < 1:
            raise ValueError(f'Cannot have zero or negative values of force_step_size '
                             f'(force_step_size={force_step_size}).')

        self.force_step_size = force_step_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.

        Parameters:
            X : array-like, shape (n_samples, n_features)
                Training data, where n_samples is the number of samples  and n_features is the number of features.

            y : array-like, shape (n_samples,)
                Always ignored, exists for compatibility.

            groups : array-like, with shape (n_samples,), optional
                Always ignored, exists for compatibility.

        Yields:
            train : ndarray
                The training set indices for that split.

            test : ndarray
                The testing set indices for that split.
        """
        X, y, groups = indexable(X, y, groups)  # pylint: disable=unbalanced-tuple-unpacking
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        delay = self.delay

        if n_folds > n_samples:
            raise ValueError(f'Cannot have number of folds={n_folds} greater than the number of samples: {n_samples}.')

        indices = np.arange(n_samples)
        split_size = n_samples // n_folds

        train_size = self.train_size or split_size * self.n_splits
        test_size = self.test_size or n_samples // n_folds
        full_test = test_size + delay

        if full_test + n_splits > n_samples:
            raise ValueError(f'test_size\\({test_size}\\) + delay\\({delay}\\) = {test_size + delay} + '
                             f'n_splits={n_splits} \n'
                             f' greater than the number of samples: {n_samples}. Cannot create fold logic.')

        # Generate logic for splits.
        # Overwrite fold test_starts ranges if force_step_size is specified.
        if self.force_step_size:
            step_size = self.force_step_size
            final_fold_start = n_samples - (train_size + full_test)
            range_start = (final_fold_start % step_size) + train_size

            test_starts = range(range_start, n_samples, step_size)

        else:
            if not self.train_size:
                step_size = split_size
                range_start = (split_size - full_test) + split_size + (n_samples % n_folds)
            else:
                step_size = (n_samples - (train_size + full_test)) // n_folds
                final_fold_start = n_samples - (train_size + full_test)
                range_start = (final_fold_start - (step_size * (n_splits - 1))) + train_size

            test_starts = range(range_start, n_samples, step_size)

        # Generate data splits.
        for test_start in test_starts:
            idx_start = test_start - train_size if self.train_size is not None else 0
            # Ensure we always return a test set of the same size
            if indices[test_start:test_start + full_test].size < full_test:
                continue
            yield (indices[idx_start:test_start],
                   indices[test_start + delay:test_start + full_test])
            
# if __name__ == '__main__':
#     X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
#     y = np.array([1, 2, 3, 4, 5, 6])
#     tscv = TimeSeriesSplit(n_splits=5)
#     print(tscv)  # doctest: +NORMALIZE_WHITESPACE
#     for train_index, test_index in tscv.split(X):
#         print('TRAIN:', train_index, 'TEST:', test_index)
#         X_train, X_test = X[train_index], X[test_index]
#         y_train, y_test = y[train_index], y[test_index]
#     print("---------------------------------------------")
#     LARGE_IDX = np.arange(0, 30)
#     rolling_window = TimeSeriesSplit(train_size=10, test_size=5, delay=3)
#     print(rolling_window)
#     for train_index, test_index in rolling_window.split(LARGE_IDX):
#         print('TRAIN:', train_index, 'TEST:', test_index)
#         X_train, X_test = LARGE_IDX[train_index], LARGE_IDX[test_index]


In [ ]:
class lightGBM(object):
    def __init__(self, learning_rate=0.05, n_estimators=20, num_boost_round=10, num_leaves=30):
        self.learning_rate = learning_rate
        self.n_estimators = n_estimators
        self.num_boost_round = num_boost_round
        self.num_leaves = num_leaves
        
        
    def fit(self, X, y):
        try:
            self.model = lgb(boosting_type='gbdt', num_leaves=self.num_leaves, learning_rate=self.learning_rate)
            self.model.fit(X, y)
        except Exception as err:
            raise Exception('Nao foi possivel treinar o modelo ', err)
        return self
    
    def predict(self,X):
        self.predicted = self.model.predict(X)
        self.predicted = pd.DataFrame(self.predicted, index= X.index, columns=['predicted'])
        return self
            

In [ ]:
# curva roc 

def display_roc_curve(model, X_test, y_test):
    plt.figure()
    lw=2 
    
#     y_pred_proba = model.predict_proba(X_test)[::,1]
    y_pred_proba = model.predict_proba(X_test)[::,1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    auc_roc = roc_auc_score(y_test, y_pred_proba)
    
    plt.plot([0,1], [0,1], color='navy', lw=lw, linestyle= '--')
    plt.plot(fpr, tpr, label= 'ROC curve(area = %0.2f)' % auc_roc, color = 'darkorange')
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title('ROC')
    plt.legend(loc='lower right')
    
    plt.show()
    

In [ ]:
# lgbm = lightGBM()

In [ ]:
# train['action'].value_counts(normalize=True)

In [ ]:
# X.shape

In [ ]:
# Feature importance

#lightGBM model fit
gbm = lgb()
gbm.fit(X, y)
gbm.booster_.feature_importance()

# importance of each attribute
fea_imp_ = pd.DataFrame({'cols':X.columns, 'fea_imp':gbm.feature_importances_})
fea_imp_ = fea_imp_.loc[fea_imp_.fea_imp > 0].sort_values(by=['fea_imp'], ascending = False)

In [ ]:
fea_imp_.head(100)

In [ ]:
train_f = X[fea_imp_.head(30).cols]

In [ ]:
del(fea_imp_)

In [ ]:
#Recursive Feature Elimination(RFE)

# from sklearn.feature_selection import RFE
# # create the RFE model and select 10 attributes
# rfe = RFE(gbm, 10)
# rfe = rfe.fit(X_train, y_train)

# # summarize the selection of the attributes
# print(rfe.support_)

# # summarize the ranking of the attributes
# fea_rank_ = pd.DataFrame({'cols':X_train.columns, 'fea_rank':rfe.ranking_})
# fea_rank_.loc[fea_rank_.fea_rank > 0].sort_values(by=['fea_rank'], ascending = True)

In [ ]:
# fea_rank_ = fea_rank_.loc[fea_rank_.fea_rank > 0].sort_values(by=['fea_rank'], ascending = True)

In [ ]:
# fea_rank_.head(10).cols

In [ ]:
rolling_window = TimeSeriesSplit(train_size=100000, test_size=5000, delay=0, n_splits=10)
# rolling_window = TimeSeriesSplit(n_splits=5)

fold = 0
# X= train_f[fea_rank_.head(10).cols]
X = train_f
clf = lgb(boosting_type='gbdt', objective='binary')


for train_index, test_index in rolling_window.split(X):
        fold +=1
#         print('TRAIN:', train_index, 'TEST:', test_index)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        
        display_roc_curve(clf, X_test, y_test)
        

In [ ]:
# clf = lgb(boosting_type='gbdt', objective='binary')
# clf.fit(X,y)
# clf.predict(example_test)

In [ ]:
# import numpy as np
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import SVC

# rolling_window = TimeSeriesSplit(train_size=10000, test_size=1000, delay=0)

# # clf = make_pipeline(StandardScaler(), SVC(gamma='scale', C=0.5, kernel='rbf', degree=5, probability=True))
# clf = SVC(gamma='scale', C=0.5, kernel='rbf', degree=5, probability=True)

# X = X.fillna(X.mean())

# for train_index, test_index in rolling_window.split(X):
#         fold +=1
#         X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#         y_train, y_test = y[train_index], y[test_index]
#         model = clf.fit(X_train, y_train)
#         pred = model.predict(X_test)
        
#         display_roc_curve(model, X_test, y_test)
        


In [ ]:
import janestreet

janestreet.competition.make_env.__called__ = False
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set
features = X.columns
for (test_df, sample_prediction_df) in iter_test:
    X_test = test_df[features]
#     for i in features:
#         x = X_test[i].mean()     
#         X_test[i] = X_test[i].fillna(x)
    X_test = X_test.fillna(0)
    y_preds = clf.predict(X_test)
    sample_prediction_df.action = y_preds
    env.predict(sample_prediction_df)